In [9]:
import numpy as np

def slug_model(pars):
    t = np.array([5.0, 10.0, 20.0, 30.0, 40.0, 50.0])
    d = 10
    Q = 50
    S, T = pars
    h_pred = (Q / (4 * np.pi * T * t)) * np.exp((-d**2 * S) / (4 * T * t))
    h_obs = np.array([0.55, 0.47, 0.30, 0.22, 0.17, 0.14])
    res = h_pred - h_obs
    SSR = np.dot(res, res)
    return SSR

def reflect(value, lower, upper):
    return lower + abs(lower - value) if value < lower else upper - abs(value - upper)

def ensure_feasible(offspring, bounds):
    for i in range(offspring.shape[0]):
        for j in range(offspring.shape[1]):
            offspring[i, j] = reflect(offspring[i, j], bounds[j][0], bounds[j][1])
    return offspring

def mutate_and_crossover(parents, F, CR, bounds):
    pop_size, dim = parents.shape
    new_population = np.zeros_like(parents)
    for i in range(pop_size):
        idxs = [idx for idx in range(pop_size) if idx != i]
        a, b, c = parents[np.random.choice(idxs, 3, replace=False)]
        mutant = a + F * (b - c)
        trial = np.array([mutant[j] if np.random.rand() < CR else parents[i, j] for j in range(dim)])
        new_population[i] = trial
    return ensure_feasible(new_population, bounds)

def initialize_population(pop_size, dim, bounds):
    return np.random.rand(pop_size, dim) * (np.array([b[1] for b in bounds]) - np.array([b[0] for b in bounds])) + np.array([b[0] for b in bounds])

def differential_evolution(cost_func, bounds, pop_size, max_gen, F, CR):
    dim = len(bounds)
    population = initialize_population(pop_size, dim, bounds)
    fitness = np.array([cost_func(ind) for ind in population])
    best_idx = np.argmin(fitness)
    best_value = fitness[best_idx]

    for _ in range(max_gen):
        new_population = mutate_and_crossover(population, F, CR, bounds)
        for i in range(pop_size):
            child_fitness = cost_func(new_population[i])
            if child_fitness < fitness[i]:
                population[i] = new_population[i]
                fitness[i] = child_fitness
                if child_fitness < best_value:
                    best_value = child_fitness
                    best_idx = i

    return population[best_idx], best_value

# DE parameters
pop_size = 20
F = 0.8  # Mutation factor
CR = 0.9  # Crossover probability
max_gen = 100  # Number of generations
bounds = [(0.001, 0.01), (1, 100)]  # Bounds for S and T

# Run the Differential Evolution algorithm
optimal_params, optimal_cost = differential_evolution(slug_model, bounds, pop_size, max_gen, F, CR)
print(f"Optimized Slug Model Parameters: {optimal_params}")
print(f"Optimal SSR: {optimal_cost}")



Optimized Slug Model Parameters: [0.01       1.11397411]
Optimal SSR: 0.06920637141977033
